In [1]:
import scipy.io # To use the '.mat' files
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# Transformada Rápida de Fourier
from scipy.fft import fft, fftfreq
# Filtrado Pasa Bajo
from scipy.signal import butter, filtfilt
# Validación cruzada
from sklearn.model_selection import cross_val_score

In [2]:
# Importar el archivo CSV con los datos filtrados (Pasa Bajo)
df = pd.read_csv('all_faults_filtered.csv')

# Obtener tipos de fallos únicos
faults = df['fault'].unique()

# Frecuencia de muestreo y frecuencia de corte para el filtro
fs = 48000  # Por ejemplo, 48 kHz
fc = 2000   # Frecuencia de corte, ajustar según sea necesario

print("Fallas:",faults)

Fallas: ['7IR' '7B' '7OR6' '7OR3' '7OR12' '14IR' '14B' '14OR' '21IR' '21B' '21OR6'
 '21OR3' '21OR21' 'N']


In [3]:

# Obtener los valores únicos de la columna 'fault'
faults = df['fault'].unique()

# Crear una lista vacía para guardar los arreglos X
X_list = []

# Crear un bucle para cada valor de 'fault'
for f in faults:
    # Obtener los datos del tipo de fallo específico
    data = df[df['fault'] == f]['DE_data'].values
    # Aplicar la Transformada Rápida de Fourier (FFT)
    fft_result = fft(data)
    # Calcular la frecuencia correspondiente
    freq = fftfreq(len(fft_result), d=1/fs)
    # Apilar las frecuencias y las amplitudes como columnas
    X = np.column_stack((freq, np.abs(fft_result)))
    # Añadir el arreglo X a la lista
    X_list.append(X)

# Apilar verticalmente los arreglos X de la lista
X = np.vstack(X_list)

# Arreglo de Y
Y = df['fault'].to_numpy().reshape(-1, 1)
Y = Y.ravel()

In [4]:
print("X Shape: (",X.shape[0],",",X.shape[1],")")
print("Y Shape: (",Y.shape[0],")")

X Shape: ( 2782629 , 2 )
Y Shape: ( 2782629 )


In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

# Suponemos que 'DE_data' es la columna con los datos de la FFT

# Si 'DE_data' es una columna con datos complejos, calcula la magnitud
# Si 'DE_data' ya contiene datos reales (por ejemplo, magnitudes de FFT), puedes omitir este paso
fft_magnitude = np.abs(df['DE_data'])

# Escalar las magnitudes de las señales FFT
scaler = StandardScaler()
scaler.fit(fft_magnitude.values.reshape(-1, 1))
scaled_features = scaler.transform(fft_magnitude.values.reshape(-1, 1))

# Crear un nuevo DataFrame con las características escaladas
df_feat = pd.DataFrame(scaled_features, columns=['DE_data_scaled'])

# Añade la columna de etiquetas al DataFrame escalado
df_feat['fault'] = df['fault']

# Ver las primeras filas para verificar
df_feat.head()

,DE_data_scaled,fault
0,-0.974651,7IR
1,-0.815748,7IR
2,-0.644107,7IR
3,-0.458021,7IR
4,-0.256859,7IR


In [6]:
print(df.columns)

Index(['DE_data', 'fault'], dtype='object')


In [7]:
#X = df['DE_data'].to_numpy().reshape(-1, 1)
#Y = df['fault'].to_numpy().reshape(-1, 1)
#Y = Y.ravel()

# Separar datos
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.75)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=10)

In [10]:
pred = knn.predict(X_test)
len(pred)

695658

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Calcula la matriz de confusión
cm = confusion_matrix(Y_test, Y_pred, normalize='true')

# Visualiza la matriz de confusión
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='.2f')
plt.xlabel('Predicciones')
plt.ylabel('Valores Verdaderos')
plt.show()


NameError: name 'Y_pred' is not defined

In [8]:
print(classification_report(Y_test,pred))

NameError: name 'classification_report' is not defined

In [12]:
%%time
knn.fit(X_train, Y_train)

CPU times: total: 8.19 s
Wall time: 8.22 s


KNeighborsClassifier(n_neighbors=10)

In [13]:
%%time
knn.score(X_train, Y_train)

CPU times: total: 1min 38s
Wall time: 1min 40s


0.3244989987881959

In [13]:
%%time
knn.score(X_test, Y_test)

CPU times: total: 35.1 s
Wall time: 35.7 s


0.10271426476803257

In [14]:
from sklearn.metrics import precision_score, recall_score

In [15]:
Y_pred = knn.predict(X_test)

In [16]:
# Calcular la precisión
precision = precision_score(Y_test, Y_pred, average='macro')  # Usa 'micro', 'macro', 'weighted' o None para problemas multiclase

# Calcular el recall
recall = recall_score(Y_test, Y_pred, average='macro')  # Usa 'micro', 'macro', 'weighted' o None para problemas multiclase

# Imprimir los resultados
print("Precisión: {:.5f}".format(precision))
print("Recall: {:.5f}".format(recall))

Precisión: 0.11904
Recall: 0.11576
